## Setup MySQL connection

In [2]:
import dotenv
import pandas as pd
import requests
from sqlalchemy.orm import Session

from persist.mysql import *
# import to create table
from persist.notam import *

dotenv.load_dotenv("../.env")

True

In [3]:
engine = MySQLEngine()

2023-05-18 16:34:26,004 INFO sqlalchemy.engine.Engine SELECT DATABASE()


INFO:sqlalchemy.engine.Engine:SELECT DATABASE()


2023-05-18 16:34:26,008 INFO sqlalchemy.engine.Engine [raw sql] {}


INFO:sqlalchemy.engine.Engine:[raw sql] {}


2023-05-18 16:34:26,611 INFO sqlalchemy.engine.Engine SELECT @@sql_mode


...

INFO:sqlalchemy.engine.Engine:DESCRIBE `notam_ai`.`notam_tag`


2023-05-18 16:34:27,840 INFO sqlalchemy.engine.Engine [raw sql] {}


INFO:sqlalchemy.engine.Engine:[raw sql] {}


2023-05-18 16:34:28,146 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


In [ ]:
# create the tables
Base.metadata.create_all(engine.get_engine())

## Get the NOTAMS from API

In [ ]:
locations = "ELLX,LFPG,EHAM"

notam_url = f"http://api.anbdata.com/anb/states/notams/notams-realtime-list?api_key={os.getenv('NOTAM_API_KEY')}&locations={locations}"

In [ ]:
res = requests.get(notam_url).json()
len(res)

In [ ]:
top20 = res[:20]
notams = [NOTAM(notam_id=notam['id'],
                message=notam['message'],
                location=notam['location']) for notam in top20]

In [ ]:
print(f'notam_id: {notams[1].notam_id}\n'
      f'message: {notams[1].message}\n'
      f'location: {notams[1].location}')

## Persist the NOTAMs to MySQL

In [ ]:
with Session(engine.get_engine()) as session:
    session.add_all(notams)

In [ ]:
with Session(engine.get_engine()) as session:
    notam = session.query(NOTAM).first()
    print(notam)

## Process the CSV Tags and persist to MySQL

In [ ]:
tags = pd.read_csv('../data/tags.csv')
tags.head(20)

In [ ]:
tags.drop(columns=[tags.columns[0], tags.columns[3]], axis=1, inplace=True)
tags.dropna(how='any', inplace=True)
tags

In [ ]:
# build notam tags model to persist
NOTAMTags = []
for index, row in tags.iterrows():
    print(row[0], row[1], row[2])
    NOTAMTags.append(NOTAMTag(level=row[0], tag=row[1], description=row[2]))

In [ ]:
# save the tags to MySQL
with Session(engine.get_engine(), expire_on_commit=False) as session:
    session.add_all(NOTAMTags)
    session.commit()

In [ ]:
with Session(engine.get_engine()) as session:
    tag = session.query(NOTAMTag).first()
    print(tag)

## LangChain LLM Integration

In [3]:
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    AIMessage,
    SystemMessage,
    HumanMessagePromptTemplate
)

## Try setup Tags as System prompts
Chat Models takes a list of chat messages as input - this list commonly referred to as a prompt. These chat messages differ from raw string (which you would pass into a LLM model) in that every message is associated with a role.

In [12]:
librarian_prompt = SystemMessage(
    content="You are a NOTAM Librarian. I will give you a series of NOTAM messages and a list of NOTAM Tags."
            "For each series of NOTAM's, create a beautiful table with the columns below. "
            "Each NOTAM should be on one row.Columns:"
            "A. **NOTAM** - NOTAM ICAO code and ID. Add one asterisk (*) before and after the NOTAM."
            "B. **Explained** - In very simple English only, explain the NOTAM in 4 words or less. Do not use abbreviations. Use sentence case."
            "C. **Tag**. Choose the most logical Tag for this NOTAM from the list of Tags. Format as Tag Code - Tag Name. "
            "Add two asterisks (**) before and after the Tag.")

notam_tags_message = HumanMessagePromptTemplate.from_template("List of NOTAM Tags,in three columns:\n"
                                                              "Tag Code  Tag Name  Tag Description\n"
                                                              "{tags}\n"
                                                              "Read and wait, no action yet.")
ai_wait_resp = AIMessage(content="Understood. Waiting for the NOTAM messages.")
notam_messages = HumanMessagePromptTemplate.from_template("{notams}")
# build the LLMChain with the chat prompts
notam_chat_prompt = ChatPromptTemplate.from_messages(
    [librarian_prompt, notam_tags_message, ai_wait_resp, notam_messages])


In [6]:
# query the tags from MySQL
with Session(engine.get_engine()) as session:
    notam_tags = session.query(NOTAMTag).all()

2023-05-18 14:59:15,336 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2023-05-18 14:59:15,344 INFO sqlalchemy.engine.Engine SELECT notam_tag.id AS notam_tag_id, notam_tag.level AS notam_tag_level, notam_tag.tag AS notam_tag_tag, notam_tag.description AS notam_tag_description, notam_tag.class_name AS notam_tag_class_name 
FROM notam_tag


INFO:sqlalchemy.engine.Engine:SELECT notam_tag.id AS notam_tag_id, notam_tag.level AS notam_tag_level, notam_tag.tag AS notam_tag_tag, notam_tag.description AS notam_tag_description, notam_tag.class_name AS notam_tag_class_name 
FROM notam_tag


2023-05-18 14:59:15,347 INFO sqlalchemy.engine.Engine [generated in 0.00379s] {}


INFO:sqlalchemy.engine.Engine:[generated in 0.00379s] {}


2023-05-18 14:59:15,648 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


In [7]:
tags_content = ""
for tag in notam_tags:
    tags_content += f"{tag.level} *{tag.tag}* {tag.description}\n"

print(tags_content)

C1 *ATC status/hours* ATC operating hours, ATC Strike, ATC failure (ATC Zero). Including FIS/AFIS
C2 *ATC procedure* TWR/APP/ACC change of procedure, lost comms procedure, contingency, emergency, DCL departure clearance
C3 *ATC flow and delay* Flow control, enroute delays, expect holding
C4 *Radio* HF, VHF, CPDLC, Satcom, ATIS - u/s, freq changes etc. KHZ, MHZ.
C5 *Radar & ADS* Radar (PSR, MSSR, SMR, PAR, TAR), ADS (ADS-B, ADS-C) & MLAT
C6 *Met* Met service hours, VOLMET, Met Equipment, Met Strikes
P1 *Airport status/hours* Airport Closed, Airport operating hours, AD AP not available
P2 *Airport restriction* Not available as alternate, airport slots, PPR required, max aircraft weight, etc.
P3 *Fire & Rescue* RFF Category change, Rescue equipment
P4 *Fuel* All fuel related NOTAMs, JET, JETA1, Avgas, Hydrants, Tankering
P5 *Apron & Parking* Apron, Stands, Gates, Followme, Apron lighting, docking, guidance, limited parking
P6 *Airport Facilities* Equipment (GSE, Rwy/Twy equip, WDI etc.), 

In [8]:
# query the notams from MySQL
with Session(engine.get_engine()) as session:
    notams = session.query(NOTAM).offset(6).limit(10).all()

2023-05-18 14:59:32,339 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2023-05-18 14:59:32,343 INFO sqlalchemy.engine.Engine SELECT notam.id AS notam_id_1, notam.notam_id AS notam_notam_id, notam.message AS notam_message, notam.location AS notam_location, notam.tag AS notam_tag, notam.summary AS notam_summary, notam.create_time AS notam_create_time, notam.update_time AS notam_update_time 
FROM notam 
 LIMIT %(param_1)s, %(param_2)s


INFO:sqlalchemy.engine.Engine:SELECT notam.id AS notam_id_1, notam.notam_id AS notam_notam_id, notam.message AS notam_message, notam.location AS notam_location, notam.tag AS notam_tag, notam.summary AS notam_summary, notam.create_time AS notam_create_time, notam.update_time AS notam_update_time 
FROM notam 
 LIMIT %(param_1)s, %(param_2)s


2023-05-18 14:59:32,344 INFO sqlalchemy.engine.Engine [generated in 0.00151s] {'param_1': 6, 'param_2': 10}


INFO:sqlalchemy.engine.Engine:[generated in 0.00151s] {'param_1': 6, 'param_2': 10}


2023-05-18 14:59:32,549 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


In [9]:
# build the notam messages
notam_messages_to_ask = ""
for notam in notams:
    notam_messages_to_ask += f"{notam.location} {notam.notam_id} {notam.message}\n"

print(notam_messages_to_ask)

ELLX A1833/23 RWY 06/24 CLSD DUE TO WIP EXC FOR FLT WITH SPECIAL PERMISSION 
FROM ANA
CREATED: 02 May 2023 09:33:00 
SOURCE: EUECYIYN
ELLX A1748/23 TWY D1 AND TWY D2, STOP BARS U/S DUE TO WIP.
CREATED: 26 Apr 2023 14:55:00 
SOURCE: EUECYIYN
ELLX A1747/23 RWY 24 CAT II/III APPROACH LGT SWITCHED OFF DUE TO WIP.
CREATED: 26 Apr 2023 14:53:00 
SOURCE: EUECYIYN
ELLX A1745/23 APRONS P5, P7, P10, CL LGT SWITCHED OFF DUE TO WIP.
CREATED: 26 Apr 2023 14:50:00 
SOURCE: EUECYIYN
ELLX A1744/23 ALL TWYS, CL LGT SWITCHED OFF DUE TO WIP.
CREATED: 26 Apr 2023 14:41:00 
SOURCE: EUECYIYN
ELLX A1742/23 RWY24, TDZ LGT SWITCHED OFF DUE TO WIP.
CREATED: 26 Apr 2023 14:28:00 
SOURCE: EUECYIYN
ELLX A1741/23 RWY 06/24, RCLL SWITCHED OFF DUE TO WIP
CREATED: 26 Apr 2023 14:15:00 
SOURCE: EUECYIYN
ELLX A1710/23 IFR TRAINING FLT NOT AUTH DUE TO CAPACITY REGULATIONS.
REF AIP BELGIUM AND LUXEMBOURG AD 2.20 ELLX LOCAL AERODROME 
REGULATIONS PARA 5.7 TRAINING AND TEST FLIGHTS
CREATED: 24 Apr 2023 20:00:00 
SOURCE: EUE

In [13]:
chat = ChatOpenAI(temperature=0)
notam_chain = LLMChain(llm=chat,
                       prompt=notam_chat_prompt)

In [35]:
response = chat(notam_chat_prompt.format_prompt(tags=tags_content, notams=notam_messages_to_ask).to_messages())

In [36]:
print(response.content)

| **NOTAM** | **Explained** | **Tag** |
|-----------|---------------|----------|
| *ELLX A1833/23* | Runway closed due to WIP | R1 - Runway closed |
| *ELLX A1748/23* | Stop bars u/s due to WIP on TWY D1 and TWY D2 | T3 - Taxiway lights |
| *ELLX A1747/23* | CAT II/III approach lights switched off due to WIP on RWY 24 | A4 - Runway lights |
| *ELLX A1745/23* | Apron lights switched off due to WIP on P5, P7, P10 | P5 - Apron & Parking |
| *ELLX A1744/23* | Centerline lights switched off due to WIP on all TWYs | T4 - Taxiway lights |
| *ELLX A1742/23* | TDZ lights switched off due to WIP on RWY 24 | A4 - Runway lights |
| *ELLX A1741/23* | Runway centerline lights switched off due to WIP on RWY 06/24 | A4 - Runway lights |
| *ELLX A1710/23* | IFR training flights not authorized due to capacity regulations | P1 - Airport status/hours |
| *ELLX A1317/23* | Outbound flights must check EOBT and update via handling or ARO if necessary | L5 - Flight Planning |
| *ELLX A1316/23* | Clearance del

| **NOTAM** | **Explained** | **Tag** |
|-----------|---------------|----------|
| *ELLX A1833/23* | Runway closed due to WIP | R1 - Runway closed |
| *ELLX A1748/23* | Stop bars u/s due to WIP on TWY D1 and TWY D2 | T3 - Taxiway lights |
| *ELLX A1747/23* | CAT II/III approach lights switched off due to WIP on RWY 24 | A4 - Runway lights |
| *ELLX A1745/23* | Apron lights switched off due to WIP on P5, P7, P10 | P5 - Apron & Parking |
| *ELLX A1744/23* | Centerline lights switched off due to WIP on all TWYs | T4 - Taxiway lights |
| *ELLX A1742/23* | TDZ lights switched off due to WIP on RWY 24 | A4 - Runway lights |
| *ELLX A1741/23* | Runway centerline lights switched off due to WIP on RWY 06/24 | A4 - Runway lights |
| *ELLX A1710/23* | IFR training flights not authorized due to capacity regulations | P1 - Airport status/hours |
| *ELLX A1317/23* | Outbound flights must check EOBT and update via handling or ARO if necessary | L5 - Flight Planning |
| *ELLX A1316/23* | Clearance delivery operational hours for Luxembourg delivery | C1 - ATC status/hours |

In [16]:
notam_table = notam_chain.run({
    "tags": tags_content,
    "notams": notam_messages_to_ask
})

In [15]:
print(notam_table)

| **NOTAM** | **Explained** | **Tag** |
|-----------|---------------|----------|
| *ELLX A1833/23* | Runway closed due to WIP | R1 - Runway closed |
| *ELLX A1748/23* | Stop bars u/s due to WIP on TWY D1 and TWY D2 | T4 - Taxiway lights |
| *ELLX A1747/23* | CAT II/III approach lights switched off due to WIP on RWY 24 | A4 - Approach lights |
| *ELLX A1745/23* | Apron lights switched off due to WIP on P5, P7, P10 | P5 - Apron & Parking |
| *ELLX A1744/23* | All taxiway centerline lights switched off due to WIP | T4 - Taxiway lights |
| *ELLX A1742/23* | TDZ lights switched off due to WIP on RWY 24 | A4 - Approach lights |
| *ELLX A1741/23* | Runway centerline lights switched off due to WIP on RWY 06/24 | R4 - Runway lights |
| *ELLX A1710/23* | IFR training flights not authorized due to capacity regulations | C2 - ATC procedure |
| *ELLX A1317/23* | Outbound flights must check EOBT and update via handling or ARO | L5 - Flight Planning |
| *ELLX A1316/23* | Clearance delivery operationa

# Speed Improvements Research

In [1]:
from app import *

In [4]:
notam_llm_chat = NOTAMLLMChat()
notam_tags = get_list_of_tags(engine.get_engine())


2023-05-18 16:34:49,067 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2023-05-18 16:34:49,073 INFO sqlalchemy.engine.Engine SELECT notam_tag.id AS notam_tag_id, notam_tag.level AS notam_tag_level, notam_tag.tag AS notam_tag_tag, notam_tag.description AS notam_tag_description, notam_tag.class_name AS notam_tag_class_name 
FROM notam_tag


INFO:sqlalchemy.engine.Engine:SELECT notam_tag.id AS notam_tag_id, notam_tag.level AS notam_tag_level, notam_tag.tag AS notam_tag_tag, notam_tag.description AS notam_tag_description, notam_tag.class_name AS notam_tag_class_name 
FROM notam_tag


2023-05-18 16:34:49,075 INFO sqlalchemy.engine.Engine [generated in 0.00169s] {}


INFO:sqlalchemy.engine.Engine:[generated in 0.00169s] {}


2023-05-18 16:34:49,357 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


In [9]:
options = [
    "V0230/23",
    # "A2072/23",
    # "A2068/23",
    # "A1949/23",
    # "A1882/23",
    # "A1834/23",
    # "A1833/23",
    # "A1748/23",
    # "A1747/23"
]
notam_messages = get_notam_messages(engine.get_engine(), options)

2023-05-18 16:46:06,070 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2023-05-18 16:46:06,075 INFO sqlalchemy.engine.Engine SELECT notam.id AS notam_id_1, notam.notam_id AS notam_notam_id, notam.message AS notam_message, notam.location AS notam_location, notam.tag AS notam_tag, notam.summary AS notam_summary, notam.create_time AS notam_create_time, notam.update_time AS notam_update_time 
FROM notam 
WHERE notam.notam_id IN (%(notam_id_1_1)s)


INFO:sqlalchemy.engine.Engine:SELECT notam.id AS notam_id_1, notam.notam_id AS notam_notam_id, notam.message AS notam_message, notam.location AS notam_location, notam.tag AS notam_tag, notam.summary AS notam_summary, notam.create_time AS notam_create_time, notam.update_time AS notam_update_time 
FROM notam 
WHERE notam.notam_id IN (%(notam_id_1_1)s)


2023-05-18 16:46:06,081 INFO sqlalchemy.engine.Engine [cached since 463.5s ago] {'notam_id_1_1': 'V0230/23'}


INFO:sqlalchemy.engine.Engine:[cached since 463.5s ago] {'notam_id_1_1': 'V0230/23'}


2023-05-18 16:46:06,342 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


In [12]:
notam_table = notam_llm_chat.chat_to_get_notam_about(notam_tags, notam_messages)
print(notam_table)

|NOTAM|Explained|Tag|
|-----|---------|---|
|**V0230/23**|[US DOD PROCEDURAL NOTAM] INSTRUMENT APPROACH PROCEDURE AMENDMENT RNP RWY 24 INCREASE LAND MINIMA. RAISE LNAV/VNAV DA CAT D TO 1617/1700M 404 (500-1700M). CHG NO-LIGHT NOTE TO READ: WHEN ALS INOP, INCR CAT ABC RVR/VIS TO 1700M; CAT D RVR/VIS TO 1900M.|A1 - Approach not available|
